In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [65]:
# CAMBIAR, HACER RELATIVOS NO SE COMO
# Mientras poner tu propia ruta
PATH = '/home/max/workspace/NASA_Spaceapps_2024/hello_flask/data/DEF_SRDB_V5_1827_5-20241005_212220/srdb-data-V5.csv'
RELATIVE_PATH = 'hello_flask/data/DEF_SRDB_V5_1827_5-20241005_212220/srdb-data-V5.csv'
DATADIR_PATH = '/home/max/workspace/NASA_Spaceapps_2024/hello_flask/data'

to_keep = [
    'Entry_date', 
    'Quality_flag', 
    'Country', 
    'Region', 
    'Site_name', 
    'Site_ID', 
    'Study_midyear', 
    'Species', 
    'Biome', 
    'Ecosystem_type',
    'Leaf_habit', 
    'Soil_type', 
    'Soil_drainage', 
    'MAT', 
    'MAP', 
    'PET', 
    'Study_temp', 
    'Study_precip'
]

to_drop = [
    'Entry_date',
    'Quality_flag',
    'Ecosystem_type', 
    'Soil_type', 
    'PET', 
    'Study_temp', 
    'Study_precip', 
    'Site_ID'
]

# Leer el archivo CSV y seleccionar solo las columnas especificadas
df = pd.read_csv(PATH, usecols=to_keep, parse_dates=['Entry_date']).drop(columns=to_drop).dropna()

# Reorganizar las columnas para que 'Study_midyear' sea la primera
columns = ['Study_midyear'] + [col for col in df.columns if col != 'Study_midyear']
df = df[columns]

# Ordenar el DataFrame por 'Study_midyear'
df = df.sort_values(by='Study_midyear').reset_index(drop=True)

# Exportar los datos
df.to_csv(DATADIR_PATH+'/Processed/1_clean_srdb-data-V5.csv', index=False)

In [66]:
df['Country'].unique()

array(['USA', 'Congo', 'United Kingdom', 'Brazil', 'Canada', 'Venezuela',
       'Japan', 'India', 'China', 'Turkey', 'Norway', 'Denmark',
       'Thailand', 'Germany', 'UK', 'Belgium', 'Italy', 'Russia',
       'Malawi', 'Finland', 'Ireland', 'Malaysia', 'Costa Rica',
       'Australia', 'South Korea', 'Italy ', 'Czech Republic', 'Mexico',
       'Spain', 'Philippines', 'Sweden', 'Panama', 'Indonesia', 'Poland',
       'Isreal', 'Zambia', 'Argentina', 'New Zealand', 'Tunisia',
       'Mongolia', 'Kenya', 'Peru', 'Switzerland', 'Greenland', 'Hungary',
       'Taiwan', 'Cesko', 'France', 'Puerto Rico', 'Portugal', 'Vietnam',
       'Singapore', 'Guinea', 'Netherlands', 'Bennin', 'Signapore',
       'Zimbabwe', 'Czechia', 'Chile', 'Estonia', 'South Africa',
       'Austria', 'Tanzania'], dtype=object)

In [67]:
df.head()

,Study_midyear,Country,Region,Site_name,Species,Biome,Leaf_habit,Soil_drainage,MAT,MAP
0,1961.0,USA,Tennessee,Oak Ridge,Pinus echinata,Temperate,Evergreen,Dry,14.0,1500.0
1,1961.0,USA,Tennessee,Oak Ridge,Acer rubrum,Temperate,Deciduous,Dry,14.0,1500.0
2,1961.0,Congo,Yangambi,Esali,Mix,Tropical,Evergreen,Dry,24.4,1875.0
3,1961.0,USA,Tennessee,Oak Ridge,Quercus alba,Temperate,Deciduous,Dry,14.0,1500.0
4,1962.0,USA,Tennessee,Oak Ridge,Quercus alba,Temperate,Deciduous,Dry,14.0,1500.0


In [68]:
df[df['Country'] == 'Mexico']

,Study_midyear,Country,Region,Site_name,Species,Biome,Leaf_habit,Soil_drainage,MAT,MAP
1780,2006.0,Mexico,Yucatan,El Eden Ecological Preserve,Mix,Tropical,Evergreen,Dry,24.2,1650.0
1938,2007.0,Mexico,Yucatan,El Eden Ecological Preserve,Mix,Tropical,Evergreen,Dry,24.2,1650.0
2821,2011.0,Mexico,Veracruz,Estero Dulce,Pachira aquatica Aubl,Tropical,Deciduous,Wet,27.0,1425.0
2822,2011.0,Mexico,Veracruz,Estero Dulce,"Thalia geniculata L.,",Tropical,Deciduous,Wet,27.0,1425.0
2823,2011.0,Mexico,Veracruz,Laguna Chica,Pachira aquatica Aubl,Tropical,Deciduous,Wet,27.0,1425.0
2824,2011.0,Mexico,Veracruz,Laguna Chica,Cyperus giganteus,Tropical,Deciduous,Wet,27.0,1425.0
2825,2011.0,Mexico,Veracruz,Boquilla de Oro,Ficus insipida,Tropical,Deciduous,Wet,27.0,1425.0
2826,2011.0,Mexico,Veracruz,Boquilla de Oro,Pontederia sagittata,Tropical,Deciduous,Wet,27.0,1425.0


In [77]:
# Definir las funciones de agregación personalizadas
agg_funcs = {
    'Country': 'first',
    'Region': 'first',
    'Study_midyear': 'first',
    'Species': lambda x: ', '.join(x.unique()),
    'Biome': 'first',
    'Leaf_habit': 'first',
    'Soil_drainage': 'first',
    'MAT': 'mean',
    'MAP': 'mean'
}

# Agrupar por 'Site_name' y 'Study_midyear' y aplicar las funciones de agregación
grouped_df = df.groupby(['Site_name', 'Study_midyear']).agg(agg_funcs).reset_index(drop=True)

# Exportar los datos
grouped_df.to_csv(DATADIR_PATH+'/Processed/2_grouped_srdb-data-V5.csv', index=False)


In [78]:
'''
# Definir las funciones de agregación personalizadas
agg_funcs = {
    'Country': 'first',
    'Region': 'first',
    'Site_name': 'first',
    'Study_midyear': 'mean',
    'Species': lambda x: ', '.join(x.unique()),
    'Biome': 'first',
    'Leaf_habit': 'first',
    'Soil_drainage': 'first',
    'MAT': 'mean',
    'MAP': 'mean'
}

# Agrupar por 'Site_name' y aplicar las funciones de agregación
grouped_df = df.groupby('Site_name').agg(agg_funcs).reset_index(drop=True)

# Exportar los datos
grouped_df.to_csv(DATADIR_PATH+'/Processed/2_grouped_srdb-data-V5.csv', index=False)
'''

"\n# Definir las funciones de agregación personalizadas\nagg_funcs = {\n    'Country': 'first',\n    'Region': 'first',\n    'Site_name': 'first',\n    'Study_midyear': 'mean',\n    'Species': lambda x: ', '.join(x.unique()),\n    'Biome': 'first',\n    'Leaf_habit': 'first',\n    'Soil_drainage': 'first',\n    'MAT': 'mean',\n    'MAP': 'mean'\n}\n\n# Agrupar por 'Site_name' y aplicar las funciones de agregación\ngrouped_df = df.groupby('Site_name').agg(agg_funcs).reset_index(drop=True)\n\n# Exportar los datos\ngrouped_df.to_csv(DATADIR_PATH+'/Processed/2_grouped_srdb-data-V5.csv', index=False)\n"

In [79]:
grouped_df[grouped_df['Country'] == 'Mexico']

,Country,Region,Study_midyear,Species,Biome,Leaf_habit,Soil_drainage,MAT,MAP
205,Mexico,Veracruz,2011.0,"Ficus insipida, Pontederia sagittata",Tropical,Deciduous,Wet,27.0,1425.0
510,Mexico,Yucatan,2006.0,Mix,Tropical,Evergreen,Dry,24.2,1650.0
511,Mexico,Yucatan,2007.0,Mix,Tropical,Evergreen,Dry,24.2,1650.0
519,Mexico,Veracruz,2011.0,"Pachira aquatica Aubl, Thalia geniculata L.,",Tropical,Deciduous,Wet,27.0,1425.0
962,Mexico,Veracruz,2011.0,"Pachira aquatica Aubl, Cyperus giganteus",Tropical,Deciduous,Wet,27.0,1425.0


In [72]:
df[df['Country'] == 'Mexico']

,Study_midyear,Country,Region,Site_name,Species,Biome,Leaf_habit,Soil_drainage,MAT,MAP
1780,2006.0,Mexico,Yucatan,El Eden Ecological Preserve,Mix,Tropical,Evergreen,Dry,24.2,1650.0
1938,2007.0,Mexico,Yucatan,El Eden Ecological Preserve,Mix,Tropical,Evergreen,Dry,24.2,1650.0
2821,2011.0,Mexico,Veracruz,Estero Dulce,Pachira aquatica Aubl,Tropical,Deciduous,Wet,27.0,1425.0
2822,2011.0,Mexico,Veracruz,Estero Dulce,"Thalia geniculata L.,",Tropical,Deciduous,Wet,27.0,1425.0
2823,2011.0,Mexico,Veracruz,Laguna Chica,Pachira aquatica Aubl,Tropical,Deciduous,Wet,27.0,1425.0
2824,2011.0,Mexico,Veracruz,Laguna Chica,Cyperus giganteus,Tropical,Deciduous,Wet,27.0,1425.0
2825,2011.0,Mexico,Veracruz,Boquilla de Oro,Ficus insipida,Tropical,Deciduous,Wet,27.0,1425.0
2826,2011.0,Mexico,Veracruz,Boquilla de Oro,Pontederia sagittata,Tropical,Deciduous,Wet,27.0,1425.0


In [73]:
df['Country'].value_counts() > 100

Country
China           True
USA             True
Canada          True
Brazil         False
Japan          False
               ...  
Guinea         False
Portugal       False
Congo          False
Cesko          False
Netherlands    False
Name: count, Length: 63, dtype: bool

In [74]:
# Contar las ocurrencias de cada país
country_counts = df['Country'].value_counts()

# Filtrar los países que tienen más de 100 ocurrencias
countries_to_keep = country_counts[country_counts > 50].index

# Filtrar el DataFrame original para mantener solo esos países
filtered_df = df[df['Country'].isin(countries_to_keep)]

filtered_df['Country'].unique()

array(['USA', 'Brazil', 'Canada', 'Japan', 'China', 'Germany', 'Spain'],
      dtype=object)

In [75]:
# Obtener el conteo de ocurrencias de cada país
country_counts = grouped_df['Country'].value_counts()

# Mostrar el top de países con el número de ocurrencias
top_countries = country_counts.head(10)  # Puedes ajustar el número de países en el top

print(top_countries)

Country
USA               680
China             514
Canada            156
Germany            50
Brazil             43
Japan              34
Spain              19
UK                 17
Czech Republic     16
Finland            16
Name: count, dtype: int64


In [81]:
grouped_df[grouped_df['Country'] == 'USA']['Study_midyear'].count()

680